<a href="https://colab.research.google.com/github/Stevenn9981/tic_tac_toe/blob/master/tic_tac_toe_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install imageio
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-6).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already th

In [2]:
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb
import random
import os
import shutil
import tempfile
import zipfile

import pygame as pg
from pygame import gfxdraw
from pygame.locals import *

import tensorflow as tf

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.networks import q_network
from tf_agents.networks import categorical_q_network
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts

tempdir = "./"

pygame 2.1.3 (SDL 2.0.22, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
tf.version.VERSION

'2.14.0'

## Hyper-parameters

In [4]:
num_iterations = 3000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
initial_collect_episodes = 5  # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
collect_episodes_per_iteration = 1 # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 32  # @param {type:"integer"}
learning_rate = 3e-4  # @param {type:"number"}
log_interval = 10  # @param {type:"integer"}

num_eval_episodes = 50  # @param {type:"integer"}
eval_interval = 50  # @param {type:"integer"}

gamma = 0.99    # @param {type:"number"}
n_step_update = 2  # @param {type:"integer"}
num_atoms = 51  # @param {type:"integer"}
min_q_value = -5  # @param {type:"integer"}
max_q_value = 5  # @param {type:"integer"}
fc_layer_params = (100,)

BOARD_SIZE = 9 # @param {type:"integer"}

REWARD_DRAW = 0    # @param {type:"number"}
REWARD_ALIVE = -0.02    # @param {type:"number"}
REWARD_NON_ADJ = -0.02    # @param {type:"number"}

# '_' means empty position, 'O' and 'X' means two players.
REWARD_ACTIVE_TWO = 0.1 # @param {type:"number"} _OO_ or _XX_, we call it active_two
REWARD_NONACT_THREE = 0.2 # @param {type:"number"} _OOOX or _XXXO or XOOO_ or OXXX_, we call it non_active_three
REWARD_ACTIVE_THREE = 0.5 # @param {type:"number"} _OOO_ or _XXX_, we call it active_three
REWARD_WIN = 1    # @param {type:"number"} _OOOO_ or _XXXX_

## Environment

In [5]:
import gym
import numpy as np
import math
from tabulate import tabulate

from typing import Tuple, List


class TicTacToeEnv1(py_environment.PyEnvironment):
    """
    Implementation of a TicTacToe Environment based on OpenAI Gym standards
    This class is modified from https://github.com/MauroLuzzatto/OpenAI-Gym-TicTacToe-Environment
    I modified that to follow the instructions of Question 1.
    """

    def __init__(self) -> None:
        """This class contains a TicTacToe environment for OpenAI Gym

        Args:
            mode (str): render mode: human or rgb_array, which returns a text or RGB array of a picture that shows the current board
        """
        self.n_actions = BOARD_SIZE * BOARD_SIZE         # 9 * 9 grids to drop

        self._observation_spec = {'state': array_spec.BoundedArraySpec(shape=(BOARD_SIZE, BOARD_SIZE, 4), dtype=np.int_, minimum=0, maximum=1),
                                  'legal_moves': array_spec.ArraySpec(shape=(self.n_actions,), dtype=np.bool_)}

        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int_, minimum=0, maximum=80)
        self.colors = [1, 2]
        self.screen = None
        self.fields_per_side = BOARD_SIZE
        self.reset()

    def observation_spec(self):
        return self._observation_spec

    def action_spec(self):
        return self._action_spec

    def get_result(self):
        return self.result

    def if_chess_nearby(self, row, col) -> bool:
        """
        Determine whether there are chess pieces in 2 squares around the given position (row, col)

        Args:
          row (int): row index
          col (int): column index

        Returns:
          res (boolean): true, if there are
        """
        for i in range(-2, 3):
            for j in range(-2, 3):
                if 0 <= row + i < BOARD_SIZE and 0 <= col + j < BOARD_SIZE:
                    if self.board[row + i, col + j] != 0:
                        return True
        return False



    def _reset(self) -> Tuple[np.ndarray, dict]:
        """
        reset the board game and state
        """
        self.board: np.ndarray = np.zeros(
            (self.fields_per_side, self.fields_per_side), dtype=int
        )
        self.current_player = 1
        self.info = {"players": {1: {"actions": []}, 2: {"actions": []}}, "Occupied": set(), "legal_moves": np.ones((self.n_actions,), dtype=bool)}
        self.latest_action = None

        # 0 means not finished, 1 or 2 means the winner and 3 means draw
        self.result = 0

        # return self.decompose_board_to_state()
        observations_and_legal_moves = {'state': self.decompose_board_to_state(), 'legal_moves': self.info["legal_moves"]}
        return ts.restart(observations_and_legal_moves)

    def decompose_board_to_state(self):
        """
        Our state is a 9x9x4 matrix.
        The first layer is the opponent's play history, 0 means no stone, 1 means stones placed by the opponent.
        The second layer is the current player's history, 0 means no stone, 1 means stones placed by the current player.
        The third layer is the opponent's latest play-out; only one entry is 1 and the others are 0.
        The fourth layer is whether the current player is the first hand. an array that is full of 1 means yes, and 0 means no.
        """
        opponent = 2 if self.current_player == 1 else 1
        o_plays = (self.board == opponent) * 1
        c_plays = (self.board == self.current_player) * 1
        l_play = np.zeros_like(self.board)
        if self.latest_action:
            r, c = self.decode_action(self.latest_action)
            l_play[r, c] = 1
        if_first = np.full_like(self.board, (self.current_player == 1) * 1)
        return np.stack([o_plays, c_plays, l_play, if_first], axis=2)


    def _step(self, action: int) -> Tuple[np.ndarray, int, bool, dict]:
        """step function of the tictactoeEnv1

        Args:
          action (int): integer between [0, 80], each representing a field on the board

        Returns:
          state (np.array): state of 2 players' history, 0 means no stone, 1 means stones placed by the corresponding player (shape: 9x9x2).
          reward (int): reward of the currrent step
          done (boolean): true, if the game is finished
          (dict): empty dict for future game related information
        """
        action = int(action)
        if not (0 <= action < self.n_actions):
            raise ValueError(f"action '{action}' is not in action_space")

        reward = REWARD_ALIVE
        (row, col) = self.decode_action(action)

        # If the agent/player does not choose an empty square, randomly select an empty one.
        if self.board[row, col] != 0:
            if len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE:
                raise ValueError('BORAD IS FULL!')
            raise ValueError('ERROR: Not A LEGAL MOVE (NOT EMPTY)')
            # action = random.choice(list(set(range(BOARD_SIZE * BOARD_SIZE)) - self.info["Occupied"]))
            # (row, col) = self.decode_action(action)
            # reward -= 5 # assign a negative reward if the play-out position is not empty

        # According to the game rules, randomly select an adjacent position with a probability of 1/16
        if random.random() < 0.5:
            row, col = self.choose_adj_pos(row, col)

        # if len(self.info["Occupied"]) != 0 and not self.if_chess_nearby(row, col):
        #     reward += REWARD_NON_ADJ

        win = False
        if 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE and self.board[row, col] == 0:
            self.board[row, col] = self.current_player  # drop the piece on the field
            win = self._is_win(self.current_player, row, col)

            cnt_act_two = self.detect_alive_two(row, col)
            cnt_non_act_three, cnt_act_three = self.detect_three(row, col)

            reward += (cnt_act_two * REWARD_ACTIVE_TWO + cnt_non_act_three * REWARD_NONACT_THREE + cnt_act_three * REWARD_ACTIVE_THREE)

            action = row * BOARD_SIZE + col
            self.latest_action = action
            self.info["players"][self.current_player]["actions"].append(action)
            self.info["Occupied"].add(action)
            self.info['legal_moves'][action] = False

        if win:
            self.result = self.current_player
            reward += REWARD_WIN
        elif len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE: # Draw
            self.result = 3
            reward += REWARD_DRAW

        done = (win or len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE)
        self.current_player = self.current_player + 1 if self.current_player == 1 else 1
        state = self.decompose_board_to_state()

        observations_and_legal_moves = {'state': state, 'legal_moves': self.info['legal_moves']}

        if done:
            return ts.termination(observations_and_legal_moves, reward)
        else:
            return ts.transition(observations_and_legal_moves, reward)

    def detect_alive_two(self, row: int, col: int) -> int:
        """ Detect how many alive_two can obtain by this play out.

        Args:
            row (int): row of the current play
            col (int): column of the current play

        Returns:
            cnt (int): the number of alive_two
        """
        cnt = 0
        adjs = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [0, -1], [1, -1], [1, 0], [1, 1]]
        for adj in adjs:
            p1_r, p1_c = row + 2 * adj[0], col + 2 * adj[1]
            p2_r, p2_c = row - adj[0], col - adj[1]
            if 0 <= p1_r < 9 and 0 <= p1_c < 9 and 0 <= p2_r < 9 and 0 <= p2_c < 9:
                if self.board[row + adj[0], col + adj[1]] == self.current_player and self.board[p1_r, p1_c] == 0 and self.board[p2_r, p2_c] == 0:
                    cnt += 1
        return cnt

    def detect_three(self, r: int, c: int) -> tuple:
        """ Detect how many non_active_three and active_three can obtain by this play out.

        Args:
            r (int): row of the current play
            c (int): column of the current play

        Returns:
            cnt_non_act (int): the number of non_active_three
            cnt_act (int): the number of active_three
        """
        cnt_non_act = 0
        cnt_act = 0
        opponent = self.current_player + 1 if self.current_player == 1 else 1
        directions = [[0, 1], [1, 0], [1, 1], [1, -1]]

        for direct in directions:
            count = 0
            for offset in range(-2, 3):
                if 0 <= r + offset * direct[0] < 9 and 0 <= c + offset * direct[1] < 9:
                    if self.board[r + offset * direct[0], c + offset * direct[1]] == self.current_player:
                        count += 1
                        if count == 3:
                            p1_r, p1_c = r + (offset + 1) * direct[0], c + (offset + 1) * direct[1]
                            p2_r, p2_c = r + (offset - 3) * direct[0], c + (offset - 3) * direct[1]

                            p1_is_empty = (0 <= p1_r < 9 and 0 <= p1_c < 9 and self.board[p1_r, p1_c] == 0)
                            p2_is_empty = (0 <= p2_r < 9 and 0 <= p2_c < 9 and self.board[p2_r, p2_c] == 0)

                            if p1_is_empty and p2_is_empty:
                                cnt_act += 1
                            elif p1_is_empty or p2_is_empty:
                                cnt_non_act += 1
                            break
                    else:
                        count = 0

        return cnt_non_act, cnt_act

    def choose_adj_pos(self, row: int, col: int) -> tuple:
        """ Randomly select an adjacent position with equal probabilities.

        Args:
            row (int): row of the current play
            col (int): column of the current play

        Returns:
            row (int): row of the selected adjacent position
            col (int): column of the selected adjacent position
        """

        adjs = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [0, -1], [1, -1], [1, 0], [1, 1]]
        adj = random.choice(adjs)
        row, col = row + adj[0], col + adj[1]
        return row, col

    def _is_win(self, color: int, r: int, c: int) -> bool:
        """check if this player results in a winner

        Args:
            color (int): of the player
            r (int): row of the current play
            c (int): column of the current play

        Returns:
            bool: indicating if there is a winner
        """

        # check if four equal stones are aligned (horizontal, verical or diagonal)
        directions = [[0, 1], [1, 0], [1, 1], [1, -1]]

        for direct in directions:
            count = 0
            for offset in range(-3, 4):
                if 0 <= r + offset * direct[0] < 9 and 0 <= c + offset * direct[1] < 9:
                    if self.board[r + offset * direct[0], c + offset * direct[1]] == color:
                        count += 1
                        if count == 4:
                            return True
                    else:
                        count = 0

        return False

    def decode_action(self, action: int) -> List[int]:
        """decode the action integer into a colum and row value

        0 = upper left corner
        8 = lower right corner

        Args:
            action (int): action

        Returns:
            List[int, int]: a list with the [row, col] values
        """
        col = action % BOARD_SIZE
        row = action // BOARD_SIZE
        assert 0 <= col < BOARD_SIZE
        return [row, col]

    def render(self, render_mode="rgb_array") -> None:
        """Render the board
        Print a string that shows the current board, if render_mode == human,
        Return the RGB array of a figure which shows the current board.
        """
        board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=str)
        for ii in range(BOARD_SIZE):
            for jj in range(BOARD_SIZE):
                if self.board[ii, jj] == 0:
                    board[ii, jj] = "-"
                elif self.board[ii, jj] == 1:
                    board[ii, jj] = "X"
                elif self.board[ii, jj] == 2:
                    board[ii, jj] = "O"

        if render_mode == "human":
            board = tabulate(board, tablefmt="fancy_grid")
            print(board)
            print("\n")


        width = height = 400

        white = (255, 255, 255)
        line_color = (0, 0, 0)

        os.environ["SDL_VIDEODRIVER"] = "dummy"
        pg.init()

        # Set up the drawing window
        if self.screen is None:
          self.screen = pg.display.set_mode([width + 16, height + 16])

        self.screen.fill(white)
        # drawing vertical lines
        for i in range(10):
          pg.draw.line(self.screen, line_color, (width / BOARD_SIZE * i, 0), (width / BOARD_SIZE * i, height), 2)

        # drawing horizontal lines
        for i in range(10):
          pg.draw.line(self.screen, line_color, (0, height / BOARD_SIZE * i), (width, height / BOARD_SIZE * i), 2)
        pg.display.flip()

        # drawing noughts and crosses
        for i in range(BOARD_SIZE):
          for j in range(BOARD_SIZE):
            if self.board[i, j] == 1: # Draw crosses
              pg.draw.lines(self.screen, line_color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                        height / BOARD_SIZE * (i + 0.5) - 10),
                                                      (width / BOARD_SIZE * (j + 0.5) + 10,
                                                        height / BOARD_SIZE * (i + 0.5) + 10)], 3)
              pg.draw.lines(self.screen, line_color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                        height / BOARD_SIZE * (i + 0.5) + 10),
                                                        (width / BOARD_SIZE * (j + 0.5) + 10,
                                                          height / BOARD_SIZE * (i + 0.5) - 10)], 3)
            elif self.board[i, j] == 2: # Draw noughts
              pg.draw.circle(self.screen, line_color, (width / BOARD_SIZE * (j + 0.5), height / BOARD_SIZE * (i + 0.5)), 12, 3)

        board = np.transpose(
                np.array(pg.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )

        return board

In [6]:
def observation_and_action_constraint_splitter(obs):
    return obs['state'], obs['legal_moves']

## Random play test

In [7]:
# py_env = suite_gym.wrap_env(TicTacToeEnv1())
py_env = TicTacToeEnv1()
tf_env = tf_py_environment.TFPyEnvironment(py_env)

In [8]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="480" height="480" controls>
      <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return IPython.display.HTML(tag)

def create_policy_eval_video(policy, filename, fps=2):
    py_env = TicTacToeEnv1()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
      time_step = tf_env.reset()
      video.append_data(py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render())

    return embed_mp4(filename)


def create_policy_battle_video(policy1, policy2, filename, fps=2):
    py_env = TicTacToeEnv1()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
      time_step = tf_env.reset()
      video.append_data(py_env.render())
      while not time_step.is_last():
        action_step = policy1.action(time_step)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render())
        if not time_step.is_last():
          action_step = policy2.action(time_step)
          time_step = tf_env.step(action_step.action)
          video.append_data(py_env.render())

    return embed_mp4(filename)

In [9]:
random_policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(), tf_env.action_spec(),
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)
create_policy_eval_video(random_policy, "random-agent")

## Agent

In [10]:
train_py_env = TicTacToeEnv1()
eval_py_env = TicTacToeEnv1()

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [11]:
conv_layer_params = [16, 32]
action_tensor_spec = tensor_spec.from_spec(tf_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Conv layers configured with the right
# activation and kernel initializer.
def conv_layer(num_units):
  return tf.keras.layers.Conv2D(
                      filters=num_units,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)


# QNetwork consists of a sequence of Conv layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
normalization1 = tf.keras.layers.BatchNormalization()
normalization2 = tf.keras.layers.BatchNormalization()
conv_layers = [conv_layer(num_units) for num_units in conv_layer_params]
action_conv = tf.keras.layers.Conv2D(filters=4,
                    kernel_size=[1, 1], padding="same",
                    data_format="channels_last",
                    activation=tf.nn.leaky_relu)
flatten = tf.keras.layers.Flatten()
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential([normalization1] + conv_layers + [action_conv, normalization2, flatten, q_values_layer])


In [12]:
class ConvNestedLayer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwds):
        super().__init__(*args, **kwds)
        self.conv1 = tf.keras.layers.Conv2D(
                      filters=32,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)
        self.conv2 = tf.keras.layers.Conv2D(
                      filters=64,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)
        self.norm1 = tf.keras.layers.BatchNormalization()
        self.norm2 = tf.keras.layers.BatchNormalization()
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x):
        x = self.norm1(x)
        x = self.conv1(x)
        x = self.norm2(x)
        x = self.conv2(x)
        x = self.flatten(x)
        return x

In [13]:
categorical_q_net = categorical_q_network.CategoricalQNetwork(
    train_env.observation_spec()['state'],
    train_env.action_spec(),
    num_atoms=num_atoms,
    preprocessing_layers = ConvNestedLayer(),
    fc_layer_params=fc_layer_params)

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# agent = dqn_agent.DqnAgent(
#     train_env.time_step_spec(),
#     train_env.action_spec(),
#     q_network=q_net,
#     observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
#     optimizer=optimizer,
#     td_errors_loss_fn=common.element_wise_squared_loss,
#     train_step_counter=train_step_counter)


agent1 = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=tf.Variable(0))

agent2 = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=tf.Variable(0))

agent1.initialize()
agent2.initialize()

## Policy

In [15]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec(),
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)

In [16]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

def compute_avg_return_battle(environment, policy1, policy2, num_episodes=10):

  total_return_1 = 0.0
  total_return_2 = 0.0
  for _ in range(num_episodes):
    episode_return_1 = 0.0
    episode_return_2 = 0.0
    time_step = environment.reset()
    while not time_step.is_last():
      action_step = policy1.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return_1 += time_step.reward
      if not time_step.is_last():
        action_step = policy2.action(time_step)
        time_step = environment.step(action_step.action)
        episode_return_2 += time_step.reward
    total_return_1 += episode_return_1
    total_return_2 += episode_return_2

  avg_return_1 = total_return_1 / num_episodes
  avg_return_2 = total_return_2 / num_episodes
  return [avg_return_1.numpy()[0], avg_return_2.numpy()[0]]

def compute_avg_win_battle(environment, policy1, policy2, num_episodes=10):

  total_return_1 = 0.0
  total_return_2 = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    while not time_step.is_last():
      action_step = policy1.action(time_step)
      time_step = environment.step(action_step.action)
      if not time_step.is_last():
        action_step = policy2.action(time_step)
        time_step = environment.step(action_step.action)
    if environment._envs[0].get_result() == 1:
        total_return_1 += 1
    elif environment._envs[0].get_result() == 2:
        total_return_2 += 1

  avg_return_1 = total_return_1 / num_episodes
  avg_return_2 = total_return_2 / num_episodes
  return [avg_return_1, avg_return_2]

In [ ]:
print(compute_avg_return(eval_env, agent1.policy, 1))
print(compute_avg_return(eval_env, agent2.policy, 1))
print(compute_avg_return_battle(eval_env, agent1.policy, agent2.policy, 10))
print(compute_avg_win_battle(eval_env, agent1.policy, agent2.policy, 10))

4.3400006
2.3400002
[1.852, 0.536]


## Replay Buffer

In [ ]:
replay_buffer1 = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent1.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

replay_buffer2 = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent2.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

def collect_episode(environment, policy1, replay_buffer1, policy2, replay_buffer2):
    time_step = environment.reset()
    trajs_1, trajs_2 = [], []

    while not time_step.is_last():
      action_step = agent1.collect_policy.action(time_step)
      next_time_step = environment.step(action_step.action)
      traj1 = trajectory.from_transition(time_step, action_step, next_time_step)
      trajs_1.append(traj1)
      time_step = next_time_step

      if not time_step.is_last():
        action_step = agent2.collect_policy.action(time_step)
        next_time_step = environment.step(action_step.action)
        traj2 = trajectory.from_transition(time_step, action_step, next_time_step)
        trajs_2.append(traj2)
        time_step = next_time_step

    # print('Before update:')
    # for tsj in trajs_1:
    #     print(f'Policy 1: {tsj.reward}')
    # for tsj in trajs_2:
    #     print(f'Policy 2: {tsj.reward}')
    # print('After update:')

    if len(trajs_1) == len(trajs_2): # Player 2 won
        for i in range(len(trajs_1) - 1):
            trajs_1[i] = trajs_1[i].replace(reward=trajs_1[i].reward - trajs_2[i].reward)
            trajs_2[i] = trajs_2[i].replace(reward=trajs_2[i].reward - trajs_1[i + 1].reward)
        trajs_1[-1] = trajs_1[-1].replace(reward=trajs_1[-1].reward - trajs_2[-1].reward)
    else: # Player 1 won
        for i in range(len(trajs_1) - 1):
            trajs_1[i] = trajs_1[i].replace(reward=trajs_1[i].reward - trajs_2[i].reward)
            trajs_2[i] = trajs_2[i].replace(reward=trajs_2[i].reward - trajs_1[i + 1].reward)


    for tsj in trajs_1:
        # print(f'Policy 1: {tsj.reward}')
        replay_buffer1.add_batch(tsj)
    for tsj in trajs_2:
        # print(f'Policy 2: {tsj.reward}')
        replay_buffer2.add_batch(tsj)

    # print()

for _ in range(initial_collect_episodes):
    collect_episode(train_env, agent1.collect_policy, replay_buffer1, agent2.collect_policy, replay_buffer2)

dataset1 = replay_buffer1.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)
iterator1 = iter(dataset1)

dataset2 = replay_buffer2.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)
iterator2 = iter(dataset2)

## Set up the checkpointer and Policy saver

In [ ]:
def create_zip_file(dirname, base_filename):
  return shutil.make_archive(base_filename, 'zip', dirname)

In [ ]:
policy_dir_1 = os.path.join(tempdir, 'policy_1')
tf_policy_saver_1 = policy_saver.PolicySaver(agent1.policy)

policy_dir_2 = os.path.join(tempdir, 'policy_2')
tf_policy_saver_2 = policy_saver.PolicySaver(agent2.policy)

## Train the agent

In [ ]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent1.train = common.function(agent1.train)
agent2.train = common.function(agent2.train)

# Reset the train step.
agent1.train_step_counter.assign(0)
agent2.train_step_counter.assign(0)

# Evaluate the agent's policy before training.
avg_return = compute_avg_win_battle(eval_env, agent1.policy, agent2.policy, 5)
policy1_win_rate = compute_avg_win_battle(eval_env, agent1.policy, random_policy, num_eval_episodes)[0]
policy2_win_rate = compute_avg_win_battle(eval_env, random_policy, agent2.policy, num_eval_episodes)[1]
print('Before training: 1_win = {0}, 2_win = {1}, Average Return = {2}'.format(policy1_win_rate, policy2_win_rate, str(avg_return)))
returns = [avg_return]

bst1, bst2 = policy1_win_rate, policy2_win_rate

# Reset the environment.
time_step = train_env.reset()

for idx in range(num_iterations):

    # Collect a few episodes using collect_policy and save to the replay buffer.
    for _ in range(collect_episodes_per_iteration):
      if idx % 2:
          collect_episode(train_env, agent1.collect_policy, replay_buffer1, agent2.collect_policy, replay_buffer2)
      else:
          collect_episode(train_env, agent2.collect_policy, replay_buffer2, agent1.collect_policy, replay_buffer1)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator1)
    train_loss_1 = agent1.train(experience).loss
    experience, unused_info = next(iterator2)
    train_loss_2 = agent2.train(experience).loss

    step = agent1.train_step_counter.numpy()

    if step % log_interval == 0:
      print('step = {0}: loss = {1}'.format(step, (train_loss_1 + train_loss_2) / 2))

    if step % eval_interval == 0:
      avg_return = compute_avg_return_battle(eval_env, agent1.policy, agent2.policy, 5)
      policy1_win_rate = compute_avg_win_battle(eval_env, agent1.policy, random_policy, num_eval_episodes)[0]
      policy2_win_rate = compute_avg_win_battle(eval_env, random_policy, agent2.policy, num_eval_episodes)[1]
      print('step = {0}: 1_win = {1}, 2_win = {2}, Average Return = {3}'.format(step, policy1_win_rate, policy2_win_rate, str(avg_return)))
      if policy1_win_rate > bst1:
        bst1 = policy1_win_rate
        tf_policy_saver_1.save(policy_dir_1)
        policy_zip_filename = create_zip_file(policy_dir_1, os.path.join(tempdir, 'exported_policy_1'))
      if policy2_win_rate > bst2:
        bst2 = policy2_win_rate
        tf_policy_saver_2.save(policy_dir_2)
        policy_zip_filename = create_zip_file(policy_dir_2, os.path.join(tempdir, 'exported_policy_2'))
      returns.append(avg_return)

## Load the trained model

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec(),
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)

In [ ]:
saved_policy_1 = tf.saved_model.load(policy_dir_1)
print(compute_avg_win_battle(eval_env, saved_policy_1, random_policy, 200))
create_policy_eval_video(saved_policy_1, "trained-agent-1")

In [ ]:
saved_policy_2 = tf.saved_model.load(policy_dir_2)
print(compute_avg_win_battle(eval_env, saved_policy_2, random_policy, 200))
create_policy_eval_video(saved_policy_2, "trained-agent-2")

In [ ]:
create_policy_battle_video(saved_policy_1, random_policy, 'trained-agent-1-with-random')

In [ ]:
create_policy_battle_video(saved_policy_2, random_policy, 'trained-agent-2-with-random')

In [ ]:
create_policy_battle_video(saved_policy_1, saved_policy_2, 'trained-agent-battle')

## TODO



*   Check whether all functions are correct
*   Check whether the environment is correctly implement
*   Add the variant in `step` function
*   Test whether the agent always chooses an *empty* square and whether they can go to an adjacent place correctly

